In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sympy as sp
import numpy as np
from Modules.classes import Operator, Coefficient, Term, Expression
from Modules.solver import *

In [5]:
hbar = Coefficient('hbar', order=0)
omega = Coefficient('omega', order=0)
Omega_z = Coefficient('Omega_z', order=0)
g = Coefficient('g', order=1)

a = Operator('a', subspace='oscillator', is_infinite=True, mat_representation=sp.Matrix([[0, 1, 0], [0, 0, 1], [0, 0, 0]]))
a_dag = Operator('{a^\\dagger}', subspace='oscillator', is_infinite=True, mat_representation=sp.Matrix([[0, 0, 0], [1, 0, 0], [0, 1, 0]]))

a.add_commutation_relation(a_dag, 1)
a_dag.add_commutation_relation(a, -1)

X = Operator('sigma_x', mat_representation=sp.Matrix([[0, 1], [1, 0]]), subspace='spin')
Z = Operator('sigma_z', mat_representation=sp.Matrix([[1, 0], [0, -1]]), subspace='spin')
Y = Operator('tau_y', mat_representation=sp.Matrix([[0, -1j], [1j, 0]]), subspace='charge')

H = hbar * omega * a_dag * a + hbar * Omega_z * 0.5 * Z + hbar * g * (a + a_dag) * X * Y
H


hbar*omega {a^\dagger}*a + Omega_z*hbar/2 sigma_z + g*hbar a sigma_x tau_y + g*hbar {a^\dagger} sigma_x tau_y

In [9]:
H_new, subspaces = H.domain_expansion()

In [7]:
subspaces

['charge', 'spin']

In [10]:
H_new

hbar*omega {a^\dagger}*a + Omega_z*hbar/2 sigma_z + g*hbar tau_y*sigma_x a + g*hbar tau_y*sigma_x {a^\dagger}

In [15]:
H_new.terms[0].info

{'coeff': [hbar, omega, 1, 1, 1, 1, 1, 1], 'oscillator': [{a^\dagger}, a]}

In [11]:
S = get_ansatz(H_new)
S

ShapeError: Matrix size mismatch: (1, 1) * (4, 4).

0

In [ ]:
H0 = hbar * omega * a_dag * a + hbar * Omega_z * Z
V = hbar * g * (a + a_dag) * X
eq = (H0 | S) + V
eq

-hbar*omega S^{(1)} a + hbar*omega {a^\dagger} S^{(2)} + g*hbar a sigma_x + g*hbar {a^\dagger} sigma_x

In [ ]:
for key, value in eq.group_by_infinite_terms().items():
    display(value)

-hbar*omega S^{(1)} + g*hbar sigma_x

hbar*omega S^{(2)} + g*hbar sigma_x

In [7]:
sp.kronecker_product(*[Z.mat_representation, Y.corresponding_id])

Matrix([
[1, 0,  0,  0],
[0, 1,  0,  0],
[0, 0, -1,  0],
[0, 0,  0, -1]])